In [1]:
%pip install fhirpy mypy

In [2]:
from fhirpy import SyncFHIRClient

from generated.resources import Patient

FHIR_SERVER_API = "<FHIR server URL>"
FHIR_SERVER_AUTH = "<Bearer, Basic or other supported auth mechanism>"

client = SyncFHIRClient(FHIR_SERVER_API, authorization=FHIR_SERVER_AUTH)

#### Define helper function to run mypy typechecking on jupyter cells

In [3]:
from mypy import api as typechecker

def typecheck(block: str):
    mypy_error, _, _ = typechecker.run([
        '--no-color-output',
        '--no-error-summary',
        '--ignore-missing-imports',
        '-c', "\n\n".join([In[2], block])
    ])
    return mypy_error or 'OK'

#### Describe `Patient` resource using generated type definitions

In [4]:
patient = Patient(name=[{"family": "Smith", "given": ["Andrew"]}], gender="male")

client.resources("Patient").search(name=patient["name"][0]["family"]).fetch()

[<SyncFHIRResource Patient/8ddefd45-7b8f-4f40-ba8c-604f5eec4884>]

#### Make sure the resource structure is aligned to the definition

In [5]:
typecheck(In[4])

'OK'

#### Should there be any errors in the constructor arguments typecheck would fail

In [6]:
Patient(name=[{"family": "Smith", "given": ["Andrew"]}], gender=["male"])

{'name': [{'family': 'Smith', 'given': ['Andrew']}], 'gender': ['male']}

In [7]:
typecheck(In[6])

'<string>:10: error: Incompatible types (expression has type "List[str]", TypedDict item "gender" has type "str")  [typeddict-item]\n'

#### Create fhir-py resource from typed dict fields

In [8]:
client.resource("Patient", **patient).save()

#### Type-cast fhir-py resource back to the definition to work with data fields

In [9]:
patient: Patient = client.resources("Patient").search(name="Smith").first()
patient["name"]

[{'given': ['Andrew'], 'family': 'Smith'}]

In [10]:
typecheck(In[9])

'OK'